In [241]:
import tkinter as tk
import tkinter.ttk as ttk
from collections import OrderedDict
from weakref import ref
import itertools
import copy
import json

In [197]:
class ScrollFrame():
    def __init__(self, master, height=100, width=100):
        self.width = width
        self.height = height
        self.outer_frame = tk.Frame(master,relief=tk.GROOVE,width=width,height=height,bd=1)
        self.canvas = tk.Canvas(self.outer_frame)
        self.inner_frame = tk.Frame(self.canvas, width=width)
        self.scrollbar = tk.Scrollbar(self.outer_frame, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.window = self.canvas.create_window((0,0), window=self.inner_frame, anchor='nw')
        self.inner_frame.bind("<Configure>", self._on_frame_configure)
        self.canvas.bind("<Configure>", self._on_canvas_configure)
        self.canvas.bind("<MouseWheel>", self._on_mouse_scroll)
        self.items = []
        
    def add_item(self, item):
        self.items.append(item)
    
    def create_widget(self, klass, *args, **kw):
        w = klass(self.inner_frame, *args, **kw)
        self.add_item(w)
        w.bind("<MouseWheel>", self._on_mouse_scroll)
        return w
    
    def _configure_widgets(self):
        self.canvas.pack(side="left", fill=tk.BOTH)
        self.scrollbar.pack(side="right", fill="y")
        self.canvas.configure(width=self.width-self.scrollbar.winfo_width())
        #self.inner_frame.pack_configure(fill=tk.X)
        for it in self.items:
            it.pack(anchor='w', expand=True, fill=tk.X)

    def pack(self, **kw):
        self.outer_frame.pack(**kw)
        self._configure_widgets()
    
    def grid(self, **kw):
        self.outer_frame.grid(**kw)
        self._configure_widgets()            
            
    def _on_frame_configure(self, e):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"),
                              width=self.width, height=self.height)
        
    def _on_mouse_scroll(self, e):
        self.canvas.yview_scroll(-int(e.delta/120), "units")
        
    def _on_canvas_configure(self, e):
        self.canvas.itemconfig(self.window,
                               width=self._calc_window_width(e.width),)
                               #height=e.height)
    
    def _calc_window_width(self, width):
        # take 2 off the width to account for relief on buttons
        return width - self.scrollbar.winfo_width()-2
        
        

In [275]:
class TabBar(ScrollFrame):
    def register_tab(self, text, cb):
        """ Register a button with the given text and callback """
        return self.create_widget(tk.Button, text=text, command=cb)
    
class SimpleEntry(tk.Entry):
    def __init__(self, master, text="", **kw):
        tv = kw.pop('textvariable', None) or tk.StringVar(None, text)
        kw['textvariable'] = tv
        super().__init__(master, **kw)

In [281]:
class ItemFrame():
    def __init__(self, master, text):
        self.master = master
        self.frame = ttk.LabelFrame(master, text=text)
        self.items = OrderedDict()
        
    def format_value(self, v):
        return "%.2f" % v
        
    def add_item(self, name, value):
        label = tk.Label(self.frame, text=name)
        entry = SimpleEntry(self.frame)
        indicator = tk.Label(self.frame, text=self.format_value(value))
        self.items[name] = (label, entry, indicator)
        return (label, entry, indicator)
        
    def grid(self, **kw):
        self.frame.grid(**kw)
        for i, (l, e, ind) in enumerate(self.items.values()):
            l.grid(row=i, column=0)
            e.grid(row=i, column=1)
            ind.grid(row=i, column=2)
            l.config(width=12)
            e.config(width=6)
            ind.config(width=6)

In [319]:
_mv_json = """{"agitation":{"pv":0,"sp":30,"man":20,"mode":2,"error":0,"interlocked":0,"output":0},"temperature":{"pv":21.674983978271484,"sp":37,"man":0,"mode":0,"error":0,"interlocked":0,"output":100},"do":{"pv":107.49867248535156,"sp":50,"manUp":0,"manDown":50,"mode":2,"error":0,"outputUp":0,"outputDown":0},"ph":{"pv":14,"sp":7,"manUp":50,"manDown":7,"mode":2,"error":200,"outputUp":0,"outputDown":0},"pressure":{"pv":-0.00034421682357788086,"error":0},"level":{"pv":0,"error":0},"condenser":{"pv":25.133401870727539,"sp":35,"man":0,"mode":0,"error":0,"output":19.733196258544922},"maingas":{"pv":46.17816162109375,"man":5,"mode":1,"error":0,"interlocked":0},"MFCs":{"air":30.75819206237793,"co2":7.6610736846923828,"n2":7.7588958740234375,"o2":7.6193175315856934}}"""
_static_json = """{"network":{"local":false},"versions":{"rio":"V3.0.64","server":"V3.0.71","type":"Mag","size":15,"database":"V2.4","serialNumber":"01961CEE","temperatureInputs":2,"doInputs":2,"phInputs":2},"agitation":{"pvUnit":"RPM","manUnit":"%","manName":"Percent Power","pvDecimals":0,"manDecimals":1},"temperature":{"pvUnit":"°C","manUnit":"%","manName":"Main Heater Duty","pvDecimals":1,"manDecimals":1},"do":{"pvUnit":"%","manUpUnit":"%","manDownUnit":"%","manUpName":"O2","manDownName":"N2","pvDecimals":1,"manUpDecimals":1,"manDownDecimals":1},"ph":{"pvUnit":"","manUpUnit":"%","manDownUnit":"%","manUpName":"Base","manDownName":"CO2","pvDecimals":2,"manUpDecimals":1,"manDownDecimals":1},"pressure":{"pvUnit":"psi","pvDecimals":1},"level":{"pvUnit":"L","pvDecimals":1,"enableLevel":1},"condenser":{"pvUnit":"°C","manName":"Filter Oven Heater Duty","manUnit":"%","pvDecimals":1,"manDecimals":1},"maingas":{"pvUnit":"L/min","manName":"Total Flow","manUnit":"L/min","pvDecimals":2,"manDecimals":2},"mfcs":{"airPvDecimals":2,"co2PvDecimals":2,"n2PvDecimals":2,"o2PvDecimals":2}}"""

class GetMainValuesWidget(tk.Frame):
    def __init__(self, master, **kw):
        super().__init__(master, **kw)
        self.mv = json.loads(_mv_json)
        self.frames = {}
        self.setup_frames()
    
    def setup_frames(self):
        max_rows = max(len(c) for c in self.mv)
        STICKY_ALL = (tk.N, tk.S, tk.E, tk.W)
        for i, key in enumerate(sorted(mv)):
            r = i // 3
            c = i % 3
            f = ItemFrame(self, key)
            for i2, k2 in enumerate(self.mv[key]):
                item = f.add_item(k2, self.mv[key][k2])
                entry = item[1]
                entry.bind("<Return>", self._accept_cb(key, k2))
            self.frames[key] = f
            f.grid(rowspan=max_rows, column=c, row=r*max_rows, sticky=STICKY_ALL)
                
    def _accept_cb(self, k1, k2):
        def cb(event):
            self.accept_number(k1, k2)
        return cb
                
    def accept_number(self, group, key):
        f = self.frames[group]
        _, e, ind = f.items[key]
        v = e.get()
        try:
            v = "%.2f" % float(v)
        except (TypeError, ValueError):
            print("Invalid value for '%s %s':" % (group, key), repr(v))
            return
        
        ind.config(text=v)
    
    def stringify(self):
        pass


In [320]:
_server_calls_ = [
    "getMainValues",
    "getStatic",
    "getDORAValues",
    "getTrendData",
    "set",
    "clearAlarm",
    "clearAlarmsbyType",
    "clearAllAlarms",
    "getConfig"
]

def create_server_call_frame(master, call):
    return GetMainValuesWidget(master)

In [321]:
class SimulatorWindow():
    def __init__(self):
        self.height = 580
        self.width = 690
        self.root = tk.Tk()
        self.root.geometry("%dx%d"%(self.width, self.height))
        self.tablabel = ttk.Label(self.root, text="Available Server Calls:")
        self.tabbar = TabBar(self.root)
        self.tabs = {}
        self.frames = {}
        self.current = None
        self.create_tabs()
        
    def create_tabs(self):
        for call in _server_calls_:
            tab = self.tabbar.register_tab(call, lambda c=call: self.MENU_ITEM_CLICKED(c))
            frame = create_server_call_frame(self.root, call)
            self.tabs[call] = tab
            self.frames[call] = frame
            #tk.Label(it.frame, text="HELLO WORLD " + call).grid()
            
    def MENU_ITEM_CLICKED(self, item):
        if item == self.current:
            return

        if self.current is not None:
            self.frames[self.current].pack_forget()
            self.tabs[self.current].config(relief=tk.RAISED)

        self.frames[item].pack(fill=tk.BOTH)
        self.tabs[item].config(relief=tk.FLAT)
        
        self.current = item
        
    def mainloop(self):
        self.show()
        self.root.mainloop()
        
    def show(self):
        self.tablabel.pack(side="top", anchor='nw')
        self.tabbar.pack(side="left", fill='y')
        self.tabs["getMainValues"].invoke()

class HelloSimulator():
    def __init__(self):
        self.window = SimulatorWindow()
    
    def run(self):
        self.window.mainloop()

In [322]:
HelloSimulator().run()

accept number
Invalid value for 'do outputUp': 'qwrqw'


In [254]:
sticky_all = (tk.N, tk.E, tk.W, tk.S)

class Item():
    def __init__(self, master, name, value=0, isnumber=True, decimals=2):
        self.name = name
        #self.frame = tk.Frame(master)
        self.label = ttk.Label(master, text=name, width=12)
        self.textvar = tk.StringVar()
        self.entry = ttk.Entry(master, textvariable=self.textvar)
        self.report = ttk.Label(master)
        self.isnumber = isnumber
        self.fmt = "%%.%df" % decimals
        self.textvar.set(self.format_value(value))
        self.value = value
        self.update_report(value)
        
        if self.isnumber:
            self.entry.bind("<Return>", self.accept_number)
        else:
            raise NotImplementedError        
        
    def accept_number(self, e):
        val = self.input_val()
        try:
            float(val)
        except ValueError:
            self.entry.bell()
            self.textvar.set(self.value)
        else:
            self.update_value(val)
        
    def format_value(self, value):
        return self.fmt % float(value)
        
    def update_value(self, value):
        self.update_report(value)
        self.value = value

    def input_val(self):
        return self.textvar.get()
    
    def grid(self, row=1, col=1):
        #self.frame.grid(row=row, column=col)
        self.label.grid(column=1, row=row, sticky=(tk.E,))
        self.entry.grid(column=2, row=row, sticky=(tk.E, tk.W))
        self.report.grid(column=3, row=row, sticky=(tk.E,))
        return 1
        
    def update_report(self, val):
        val = self.format_value(val)
        self.report.config(text=val)

class ItemList():
    def __init__(self, master, name, fields):
        self.frame = tk.LabelFrame(master, text=name)
        self.items = OrderedDict()
        self.name = name
        for f, val in fields.items():
            self.items[f] = Item(self.frame, f, val)
            
    def grid(self, row=1, col=1, cur_row=0):
        self.frame.grid(row=row, column=col, sticky=sticky_all)
        nrows = 0
        for i, item in enumerate(self.items.values()):
            nrows += item.grid(i+cur_row, col*3)
        return nrows
    
    def iteritems(self):
        return list(self.items.values())
    
    def dictify(self):
        v = {k: item.value for k, item in self.items.items()}
        return v
            
class MainValuesFrame():
    def __init__(self, master, mv):
        self.groups = OrderedDict()
        self.frame = ttk.LabelFrame(master, text="HELLO!")
        for group in mv:
            self.groups[group] = ItemList(self.frame, group, mv[group])
        self.apply_btn = ttk.Button(self.frame, text="Apply", command=self.apply)
        self.master = master
            
    def apply(self):
        for item in self.iteritems():
            value = item.input_val()
            item.update_value(value)
    
    def grid(self):
        subcols = 3
        nrows = 0
        for i, frame in enumerate(self.groups.values()):
            nrows += frame.grid(i//subcols, i%subcols, nrows)   
        self.frame.grid(columnspan=subcols, rowspan=nrows)
        #self.frame.grid()
        self.apply_btn.grid(column=(subcols-1)//2, row=nrows+1)
        for item in self.iteritems():
            item.label.config(width=12)
            item.entry.config(width=6)
            item.report.config(width=6)
                            
    def iteritems(self):
        items = []
        for frame in self.groups.values():
             items.extend(frame.iteritems())
        return items
    
    def dictify(self):
        rv = {k: group.dictify() for k, group in self.groups.items()}
        return rv

            
        

In [255]:
import threading
import queue

class RepeatedTask():
    def __init__(self, master, func, interval):
        self.master = master
        self.func = func
        self.interval = interval
        self.after_id = None
        
    def schedule(self):
        self.after_id = self.master.after(self.interval, self.func)
    
    def cancel(self):
        if self.after_id:
            self.master.after_cancel(self.after_id)
        
    def update(self):
        try:
            self.func()
        finally:
            self.schedule()

class Window():
    def __init__(self, server=None, update_interval=0.5):
        self.tk = tk.Tk()
        self.server = server
        self.update_interval=0.5
        self.frames = OrderedDict()
        self.tasks = []
        self.server_thread = None
        self.server_running = True
        
    def set_server(self, server):
        self.server = server
        
    def mainloop(self):
        if not self.server:
            raise AttributeError("Server not set!")
        self.server_thread = threading.Thread(None, self.run_server, daemon=True)
        self.server_thread.start()
        self.tk.mainloop()
        self.server_running = False
        self.server_thread.join()
    
    def run_server(self):
        while self.server_running:
            self.server.handle_request()
    

In [164]:
import socket
import ssl
import http.server
from urllib.parse import urlparse as url_parse, parse_qs as ulib_parseqs
import os

usr = os.path.expanduser("~")
serve_path = os.path.join(usr, "Personal", "test")
SERVER_DIR = os.path.abspath(serve_path)
os.makedirs(SERVER_DIR, exist_ok=True)

class Server():
    def __init__(self, host="localhost", port=12345, listen=1):
        self.connections = []
        self.sock = self.init_sock(host, port)
        self.listen = listen
        self.configure_socket(listen)
    
    def configure_socket(self, listen):
        self.sock.listen(listen)
        
    def init_sock(self, host, port):
        s = socket.socket()
        s.bind((host, port))
        return s
    
def parse_qs(qs):
    dct = ulib_parseqs(qs)
    rv = {}
    for k, v in dct.items():
        v = v[0]
        if isinstance(v, str):
            v = v.lower()
        rv[k.lower()] = v
    return rv
    

class BadServerCall(Exception):
    def __init__(self, code=405, msg="", content_type="application/json"):
        self.code = code
        self.args = msg,
        if not isinstance(msg, bytes):
            msg = msg.encode('utf-8')
        self.msg = msg
        self.content_type = content_type
        
        
class ServerBackend():
    def __init__(self, mv_frame):
        self.mv_frame = mv_frame
        self.calls = {
            "getmainvalues": self.do_getMainvalues,
        }
    def do_getMainvalues(self, args):
        
        mv = self.mv_frame.dictify()
        mv = json.dumps(mv)
        
        # Real web server behavior...
        if "json" not in args:
            mv = "<Reply><Result>True</Result><Message>%s</Message></Reply>" % mv
            content_type = "application/xml"
        else:
            content_type = "application/json"
        return mv.encode("utf-8"), content_type

    def execute(self, call, params):
        handler = self.calls[call]
        return handler(params)
    
    
class HelloHandler(http.server.SimpleHTTPRequestHandler):
    
    # Inherited attributes and methods
    close_connection = False
    timeout = 0
    
    def setup(self):
        super().setup()
        self.backend = server.backend
        self.headers = {}
        
    def finish(self):
        pass
    
    def handle(self):
        self.handle_one_request()
        
    # Meat and potatoes
    def do_GET(self):
        parsed = url_parse(self.path)
        path = os.path.join(SERVER_DIR, parsed.path)
        if os.path.exists(path):
            self.send_file_reply(200, path)
        elif parsed.path == "/webservice/interface/":
            try:
                self.handle_servercall(parsed)
            except BadServerCall as e:
                self.send_error(e.code, e.msg)
        else:
            self.send_error(503, "ERROR 503: BAD PATH '%s'"%self.path)
            
    # server call handler
    def handle_servercall(self, parsed):
        params = parse_qs(parsed.query)
        call = params.pop("call", None)
        # Check if call is not found or is listed as ""
        if not call:
            call = params.pop("getfile", None)
            if not call:
                raise BadServerCall(405, "No Call Parameter")
        try:
            msg, content_type = self.server.backend.execute(call, params)
        except KeyError as e:
            raise BadServerCall(502, str(e))
        self.send_reply(200, msg, content_type)   

    def send_file_reply(self, code, path):
        
        extension = os.path.splitext(path)[1]
        if extension == ".png":
            content_type = "image/png"
        elif extension == ".css":
            content_type = "text/css"
        elif extension == ".html":
            content_type = "text/html"
        elif extension == ".js":
            content_type = "application/x-javascript"
        elif extension == ".gif":
            content_type = "image/gif"
        elif extension == ".ico":
            content_type = "image/x-icon"
        elif extension == ".min.map":
            content_type = "application/json"
        else:
            self.send_error(501, "Invalid Path Request: '%s'" % path)
            return
        
        with open(path, 'rb') as f:
                body = f.read()
        self.send_reply(code, body, content_type)
        
    def send_reply(self, code, body, content_type):
        if not isinstance(body, bytes):
            body = body.encode("utf-8")
        self.send_response(code)
        for h, v in self.headers.items():
            self.send_header(h, v)
        self.send_header("Content-Length", len(body))
        self.send_header("Content-Type", content_type)
        self.end_headers()
        self.wfile.write(body)
        self.wfile.flush()
        
    def send_error(self, code, msg, content_type=None):
        if isinstance(msg, str):
            msg = msg.encode("utf-8")
        content_type = content_type or "application/json"
        self.send_reply(code, msg, content_type)
    
class Server2(http.server.HTTPServer):
    timeout=0
    def __init__(self, host="localhost", port=12345, backend=None):
        super().__init__((host, port), HelloHandler)
        self.http_connection = None
        self.https_connection = None
        if not backend:
            raise ValueError("Must have backend")
        self.backend = backend
        
    def process_request(self, request, client_address):
        self.finish_request(request, client_address)
    
    

In [165]:
w = Window()
mf = MainValuesFrame(w.tk, mv)
backend = ServerBackend(mf)
server = Server2("localhost", 12345, backend)
w.set_server(server)
mf.grid()
w.mainloop()